In [ ]:
import requests
journal_code='mzm'
paper_id=4119

url_full_text = f'http://www.mathnet.ru/php/getFT.phtml?jrnid={journal_code}&paperid={paper_id}&what=fullt&option_lang=rus'
url_summary = f'http://www.mathnet.ru/php/archive.phtml?wshow=paper&jrnid={journal_code}&paperid={paper_id}&option_lang=rus'

filename_pdf = f'{journal_code}-{paper_id}.pdf'
filename_summary = f'{journal_code}-{paper_id}-resume.html'
response_text = requests.get(url_full_text)
response_summary = requests.get(url_summary)
f = open(filename_pdf, 'w+b')
f.write(response_text.content)
f.close()
f = open(filename_summary, 'w+b')
f.write(response_summary.content)
f.close()

In [1]:
import os
import re, string
import pandas as pd
import numpy as np
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from gensim.models import Phrases
from gensim.corpora.dictionary import Dictionary
from numpy import array
from gensim.models.ldamulticore import LdaMulticore
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib
from bs4 import BeautifulSoup

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lalsk\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\lalsk\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
import aspose.words as aw

In [3]:
for dirname, _, filenames in os.walk('C:/Users/lalsk/Desktop/d/статьи/test2'):
    for filename in filenames:
        print(filename)
        #files.append(filename)
        my_file = os.path.join(dirname, filename)
        doc = aw.Document(my_file)
        #doc.save(my_file + " convert" +".txt")
        doc.save('C:/Users/lalsk/Desktop/d/статьи/test2/' + filename + " convert" +".txt")

issledovanie-konechnyh-algebraicheskih-struktur-v-sisteme-maxima.pdf convert.txt


KeyboardInterrupt: 

In [19]:
articles = []
for dirname, _, filenames in os.walk('C:/Users/lalsk/Desktop/d/статьи/test2'):
    for filename in filenames:
        #print(filename)
        my_file = os.path.join(dirname, filename)
        file = open(my_file,'r',encoding = "utf-8")
        temp = file.read()
        articles.append(temp)

In [20]:
article = articles[0]

In [21]:
stop = ['это','\ufeff', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', 'км','наш','который', 'теорема', 'лемма','Общероссийский',
           'математический','портал']
stop_tags = stopwords.words('russian') + stop + stopwords.words('english')
regex = re.compile('[%s]' % re.escape(string.punctuation)) 
stemmer = SnowballStemmer("russian")

In [22]:
def preprocessing(text):
    text = text.replace('\n',' ')
    text = re.sub(r'[.,"\'-?:!;]', '', text)
    text = re.sub(r'[a-zA-Z]', '', text)
    text = re.sub(r'[λ,τ,χ,σ,ε,×,∈,−,ω,∩,⊆,≤,α,ξ,∆,η,£,•,—,✷,ω,∀,∃,ℵ,¬,β,π,σ,∼,§,❅,γ,θ,¯,→,∗,№,≺,ψ,≡,∨,ϕ,∞,δ,⊥,❁,⊕,◦,⊗,µ,∪,·,∂,∅,ρ]', '', text)
    text = re.sub(r'\s+[а-яА-Я]\s+', ' ', text)
    text = re.sub(r'\^[а-яА-Я]\s+', ' ', text)
    text = re.sub(r'\s+[а-яА-Я]\^', ' ', text)
    text = regex.sub('', text) # удаляем пунктуацию
    text = re.sub(r'^b\s+', '', text)
    text = [token for token in text.split(' ') if token not in stop_tags] # Удаляем стоп слова
    text = [stemmer.stem(token) for token in text] # Выполняем стэмминг        
    text = [token for token in text if token] # Удаляем пустые токены
    return ' '.join(text)

In [23]:
prep_article =  preprocessing(article)

In [24]:
text_clean_test= []
for j in list(prep_article.lower().split(' ')):
                text_clean_test.append(j.split())

In [25]:
bigrams_test = []
trigrams_test= []
for i in range(len(text_clean_test)):
    bigram = Phrases(text_clean_test[i]) # Создаем биграммы на основе корпуса
    trigram = Phrases(bigram[text_clean_test[i]])# Создаем триграммы на основе корпуса
    bigrams_test.append(bigram)
    trigrams_test.append(trigram)

In [26]:
for j in range(len(text_clean_test)):
    for i in range(len(text_clean_test[j])):
        for token in bigrams_test[j][text_clean_test[j][i]]:
            if '_' in token:
                # Токен это биграмма, добавим в документ.
                text_clean_test[j][i].append(token)
        for token in trigrams_test[j][text_clean_test[j][i]]:
            if '_' in token:
                text_clean_test[j][i].append(token)

In [27]:
from gensim import corpora

In [28]:
dictionary = corpora.Dictionary.load('C:/Users/lalsk/Desktop/d/1/dictionary.dict')

In [29]:
list_topics_test = []
corpus = [dictionary.doc2bow(doc) for doc in text_clean_test]
print('Количество уникальных токенов: %d' % len(dictionary))
print('Количество документов: %d' % len(corpus))
model=LdaMulticore(corpus=corpus,id2word=dictionary, num_topics=14)
temp_list = []
if (model != 0):
    for j in range(4):
        temp_list.append(model.show_topic(j,topn=14))
else: 
    temp_list.append('0')
list_topics_test.append(temp_list)

Количество уникальных токенов: 13284
Количество документов: 964


In [30]:
keywords_test = []
for i in range(len(list_topics_test)):
    temp_list = []
    for j in range(len(list_topics_test[i])):
        for k in range(len(list_topics_test[i][j])):
            temp_list.append(list_topics_test[i][j][k][0])
    keywords_test.append(temp_list)

In [31]:
keywords_t = keywords_test[0]

In [32]:
keywords_train = pd.read_csv('C:/Users/lalsk/Desktop/d/files/keywords_train.csv')

In [33]:
keywords_=keywords_train['keywords'].tolist()

In [34]:
type(keywords_[0])

str

In [35]:
for i in range(len(keywords_)):
    keywords_[i] = keywords_[i].replace('[','')
    keywords_[i] = keywords_[i].replace(']','')
    keywords_[i] = keywords_[i].replace(',','')
    keywords_[i] = re.sub(r'[''.,"\'-?:!;€'']', '', keywords_[i])

In [36]:
keywords_t = " ".join(keywords_t)

In [37]:
keywords_.append(keywords_t)

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [39]:
tfidfconverter = TfidfVectorizer(max_features=1000,max_df=0.7)

In [25]:
keywords_train_vectorized = tfidfconverter.fit_transform(keywords_).toarray()

In [26]:
df5 = pd.DataFrame(keywords_train_vectorized)
df5.dropna()

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.074340,0.0,0.0,0.000000,0.000000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.176958
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.096307,0.0,0.0,0.000000,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.192658,0.0,0.0,0.0,0.0,0.082304,0.0,0.0,0.180471,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
3126,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
3127,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
3128,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000


In [27]:
classifer = joblib.load("Rforest.pkl")

In [28]:
new_observation = []

In [29]:
new_observation.append(df5.iloc[3129].tolist())

In [30]:
classifer.predict(new_observation)

array(['512'], dtype='<U11')

In [31]:
list_topics_test

[[[('нижн', 0.0030910922),
   ('точност', 0.0030881537),
   ('коммутативн', 0.0030868624),
   ('индекс', 0.0020842555),
   ('структур', 0.0020831812),
   ('процедур', 0.0020822554),
   ('развива', 0.0010788733),
   ('классическ', 0.0010786756),
   ('полност', 0.0010786578),
   ('наложен', 0.0010780917),
   ('мультипликативн', 0.0010780878),
   ('применя', 0.0010780834),
   ('материал', 0.0010780043),
   ('произведен', 0.0010779846)],
  [('элемент', 0.00603742),
   ('под', 0.0020607465),
   ('математик', 0.0020603456),
   ('алгебр', 0.0020602474),
   ('сложен', 0.0020597542),
   ('числ', 0.002059135),
   ('структур', 0.0020590348),
   ('изоморфн', 0.0020586231),
   ('кажд', 0.0020584036),
   ('восстанов', 0.0010660773),
   ('называ', 0.0010659716),
   ('строг', 0.0010659694),
   ('подробн', 0.001065966),
   ('обратим', 0.00106594)],
  [('индекс', 0.0060384655),
   ('компьютерн', 0.0040463996),
   ('диагона', 0.0020604406),
   ('таблиц', 0.0020591647),
   ('каноническ', 0.0020588778),
  

In [32]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

In [33]:
vis = gensimvis.prepare(model, corpus, dictionary)
vis

C:\Users\lalsk\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
10    -0.009242  0.021401       1        1  9.682591
4     -0.023628 -0.011478       2        1  8.621263
13    -0.000771  0.000409       3        1  8.121486
9      0.009949  0.000542       4        1  7.780054
6      0.002988 -0.007020       5        1  7.561479
11    -0.000945  0.004667       6        1  7.466808
3     -0.001357 -0.007885       7        1  6.688708
5     -0.002563  0.002310       8        1  6.657554
2     -0.000135  0.002273       9        1  6.563927
1      0.007691 -0.004744      10        1  6.376676
8      0.005460 -0.003674      11        1  6.345351
12     0.007124  0.000301      12        1  6.221361
7      0.000653  0.004125      13        1  6.065321
0      0.004775 -0.001228      14        1  5.847422, topic_info=                 Term      Freq     Total Category  logprob  loglift
1054           операц  2.000000  2.000000  Default  30.0000  30.0000
1493          умножен  1.000000  1.000000  Default  29.0000  29.0000
2001           индекс  1.000000  1.000000  Default  28.0000  28.0000
14                  –  1.000000  1.000000  Default  27.0000  27.0000
4615            поиск  0.000000  0.000000  Default  26.0000  26.0000
...               ...       ...       ...      ...      ...      ...
103            пример  0.050527  0.282607  Topic14  -6.8332   1.1176
3277  мультипликативн  0.050558  0.308851  Topic14  -6.8326   1.0294
332              числ  0.050522  0.371474  Topic14  -6.8333   0.8441
2062        полугрупп  0.050478  0.470374  Topic14  -6.8342   0.6072
2001           индекс  0.097744  1.121974  Topic14  -6.1733   0.3987

[511 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
2001      6  0.891286   индекс
1054      1  0.719527   операц
4615      3  1.142521    поиск
1493      2  0.904837  умножен
14        5  0.838193        –, R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[11, 5, 14, 10, 7, 12, 4, 6, 3, 2, 9, 13, 8, 1])

In [35]:
keywords_test[0]

C:\Users\lalsk\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['нижн',
 'точност',
 'коммутативн',
 'индекс',
 'структур',
 'процедур',
 'развива',
 'классическ',
 'полност',
 'наложен',
 'мультипликативн',
 'применя',
 'материал',
 'произведен',
 'элемент',
 'под',
 'математик',
 'алгебр',
 'сложен',
 'числ',
 'структур',
 'изоморфн',
 'кажд',
 'восстанов',
 'называ',
 'строг',
 'подробн',
 'обратим',
 'индекс',
 'компьютерн',
 'диагона',
 'таблиц',
 'каноническ',
 'изоморфн',
 'всем',
 'понятн',
 'аддитивн',
 'петр',
 '»',
 'å',
 'информатик',
 'налож',
 '–',
 'множеств',
 'код',
 'таблиц',
 'применен',
 'каноническ',
 'полурешеточн',
 'числ',
 'есл',
 'представлен',
 'зада',
 'пуст',
 'единствен',
 'представл']

In [38]:
keywords_train['keywords'][1]

C:\Users\lalsk\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


"['групп', 'произведен', 'секц', 'сибирск', 'ран', 'гов', 'множ', 'положительн', 'последн', 'люб', 'нор', 'мальн', 'общност', 'силовск', 'следовательн', 'полож', 'ул', 'соответствен', 'котор', 'подгрупп', 'счита', 'фонд', 'рассужден', 'замен', 'номер', 'постро', 'содерж', 'редколлег', 'групп', 'нормальн', 'порядк', 'изоморфн', 'доказательств', 'конечн', 'теор', 'цеп', 'нов', 'одн', 'сил', 'секц', 'либ', 'определ', 'групп', 'нормальн', 'циклическ', 'явля', 'секц', 'произведен', 'существ', 'вопрос', 'чисел', 'аабутурлакин', 'стар', 'положительн', 'хухр', 'иском']"

In [18]:
keywords_train.head()

,Unnamed: 0,udc,keywords
0,0,517,"['подпоследовательн', 'последовательн', 'жп', ..."
1,1,512,"['групп', 'произведен', 'секц', 'сибирск', 'ра..."
2,2,517,"['х', 'отрезк', 'параграф', 'обознач', 'функц'..."
3,3,517,"['°', 'двух', 'числ', 'свойств', 'тройк', 'дос..."
4,4,510,"['метаперемен', 'множеств', 'называ', 'проверк..."
